In [2]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import numpy as np
import albumentations as A
from albumentations import Compose, HorizontalFlip, RandomRotate90, ColorJitter, RandomCrop, GaussianBlur, Normalize
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os.path
from PIL import Image, ImageDraw
from sahi.utils.file import load_json, save_json
from tqdm import tqdm

import torchvision
import torch


In [6]:
torch.cuda.is_available()
torch.cuda.current_device()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [36]:
coco_file_name = "cassette1_train"

original_file = f"{coco_file_name}_corrected_coco.json"
sliced_file = f"{coco_file_name}_sliced_coco.json"

print(original_file)
print(sliced_file)

# Define paths
DATA_DIR = os.path.join(".", "data", "coco")
ORG_ANNOTATION_PATH = os.path.join(DATA_DIR, original_file)
SLC_ANNOTATION_PATH  = os.path.join(DATA_DIR, sliced_file)
AUGMENTATION_PATH =  os.path.join("..", "data","augmentation")

IMAGE_DIR = os.path.join(DATA_DIR, "images") 
SLICED_IMAGE_DIR = os.path.join(DATA_DIR, "images_sliced",coco_file_name) 
VISUALIZATION_DIR = os.path.join(".", "data", "bbox_vis")
BBOX_VISUALISATION_DIR = os.path.join

# Ensure directories exist
os.makedirs(AUGMENTATION_PATH, exist_ok=True)

print(AUGMENTATION_PATH)
print("Original Annotation:", ORG_ANNOTATION_PATH)
print("Sliced Annotation: ", SLC_ANNOTATION_PATH)

print(IMAGE_DIR)
print(SLICED_IMAGE_DIR)


cassette1_train_corrected_coco.json
cassette1_train_sliced_coco.json
..\data\augmentation
Original Annotation: .\data\coco\cassette1_train_corrected_coco.json
Sliced Annotation:  .\data\coco\cassette1_train_sliced_coco.json
.\data\coco\images
.\data\coco\images_sliced\cassette1_train


## Full image augmentation

In [ ]:
coco_dict = load_json(ORG)
[img.update({"file_name": img["file_name"].split("/")[-1]}) for img in coco_dict["images"]]
save_json(coco_dict, save_path=NEW_AUGMENTATION_PATH)

coco_dict

FileNotFoundError: [Errno 2] No such file or directory: '.\\data\\coco\\cassette1_train.json'

In [2]:
for img in coco_dict["images"]:
    fig, ax = plt.subplots(1, 1, figsize=(12, 9), constrained_layout=True)

    mono_img = Image.open(os.path.join(IMAGE_DIR, img["file_name"])).convert("L")
    rgb_img = Image.merge("RGB", (mono_img, mono_img, mono_img))

    # iterate over all annotations
    for ann_ind in range(len(coco_dict["annotations"])):
        
        if coco_dict["annotations"][ann_ind]["image_id"] == img["id"]:
            # convert coco bbox to pil bbox
            xywh = coco_dict["annotations"][ann_ind]["bbox"]
            xyxy = [xywh[0], xywh[1], xywh[0]+xywh[2], xywh[1]+xywh[3]]

            # visualize bbox over image
            ImageDraw.Draw(rgb_img).rectangle(xyxy, width=5, outline="lime")

    ax.axis("off")
    ax.imshow(rgb_img)
    fig.savefig(os.path.join(BBOX_VISUALIZATION_DIR, img["file_name"][:-4] + ".png"))
    
    plt.clf()
    plt.cla()

NameError: name 'coco_dict' is not defined

In [15]:
class DefectAugmentationPipeline:
    def __init__(self, scale_classification):
        self.scale_classification = scale_classification
        self.augmentations_small, self.augmentations_large = self.define_augmentations()

    def define_augmentations(self):
        # Define augmentations for small and large scale
        augmentations_small = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Rotate(limit=90, p=0.5),
            A.RandomBrightnessContrast(p=0.3),
            A.GaussianBlur(blur_limit=(3, 5), p=0.3),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            ToTensorV2(),
        ], bbox_params=A.BboxParams(format='coco', label_fields=['class_labels']))

        augmentations_large = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Rotate(limit=45, p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.GaussianBlur(blur_limit=(5, 7), p=0.3),
            ToTensorV2(),
        ], bbox_params=A.BboxParams(format='coco', label_fields=['class_labels']))

        return augmentations_small, augmentations_large

    def get_scale_type(self, defect_types):
        # Determine predominant scale based on defect types
        return "small" if any(
            self.scale_classification.get(dt, "Small Scale") == "Small Scale" for dt in defect_types
        ) else "large"

    def augment_image(self, image, bboxes, class_labels, defect_types):
        # Choose augmentation based on scale
        predominant_scale = self.get_scale_type(defect_types)
        augmentation = self.augmentations_small if predominant_scale == "small" else self.augmentations_large
        
        # Apply augmentation
        augmented = augmentation(image=image, bboxes=bboxes, class_labels=class_labels)
        return augmented['image'], augmented['bboxes']

    def prepare_image(self, image_path):
        # Load and convert image to compatible format for albumentations
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError("Failed to load image. Check the file path and format.")
        return cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)  # Convert to 3-channel